The reimplementation code was adapted from the publicly available code from Mekala et al. [1]

In [ ]:
!curl -Lo prev.zip https://figshare.com/ndownloader/files/27188972

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 3867M  100 3867M    0     0  12.4M      0  0:05:10  0:05:10 --:--:-- 18.9M


In [ ]:

!unzip prev.zip   -d .

Archive:  prev.zip
 extracting: ./publish/__init__.py   
   creating: ./publish/data/
  inflating: ./publish/data/P1-Golden.xlsx  
  inflating: ./publish/data/P2-Golden.xlsx  
  inflating: ./publish/data_loader.py  
  inflating: ./publish/evaluate.py   
  inflating: ./publish/ML Baseline.ipynb  
  inflating: ./publish/models.py     
   creating: ./publish/models/
  inflating: ./publish/models/p1-bert.bin  
  inflating: ./publish/models/p1-bert-config.json  
  inflating: ./publish/models/p1-elmo.bin  
  inflating: ./publish/models/p1-fasttext.bin  
  inflating: ./publish/models/p2-bert.bin  
  inflating: ./publish/models/p2-bert-config.json  
  inflating: ./publish/models/p2-elmo.bin  
  inflating: ./publish/models/p2-fasttext.bin  
   creating: ./publish/models/pre-trained/
  inflating: ./publish/models/pre-trained/crawl-300d-2M.vec  
  inflating: ./publish/models/pre-trained/elmo_2x4096_512_2048cnn_2xhighway_5.5B_options.json  
  inflating: ./publish/models/pre-trained/elmo_2x4096_512

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 34.0 MB/s 
     |████████████████████████████████| 182 kB 78.6 MB/s 
     |████████████████████████████████| 7.6 MB 63.8 MB/s 


In [ ]:
!pip install allennlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 730 kB 5.5 MB/s 
     |████████████████████████████████| 125 kB 50.0 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
     |████████████████████████████████| 4.4 MB 54.3 MB/s 
     |████████████████████████████████| 316 kB 13.6 MB/s 
     |████████████████████████████████| 248 kB 55.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 306 kB 65.2 MB/s 
     |████████████████████████████████| 6.5 MB 69.7 MB/s 
     |████████████████████████████████| 1.8 MB 69.6 MB/s 
     |████████████████████████████████| 776.3 MB 1.3 kB/s 
     |████████████████████████████████| 880 kB 74.8 MB/s 
     |████████████████████████████████| 593 kB 63.0 MB/s 
     |████████████████████████████████

In [ ]:
! pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 4.1 MB/s 
  Using cached pybind11-2.10.1-py3-none-any.whl (216 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-linux_x86_64.whl size=3134045 sha256=b4c7c7e43c90c667c79182a444a792a9f90d09105944db2cacb9dd50d570b7ac
  Stored in directory: /root/.cache/pip/wheels/93/61/2a/c54711a91c418ba06ba195b1d78ff24fcaad8592f2a694ac94
Successfully built fasttext


# Prepare Data

In [ ]:
import pandas as pd
train_X_original = pd.read_csv("./data/original_data_X_train.csv")
train_y_original = pd.read_csv("./data/original_data_y_train.csv")
augmented_data = pd.read_csv("./data/balanced_augmentation_dataset.csv")
X_test = pd.read_csv("./data/original_data_X_test.csv")
y_test = pd.read_csv("./data/original_data_y_test.csv")

augmented_data=augmented_data[["Reviews", "Useful?"]]
augmented_data.rename(columns={"Reviews":"reviews", "Useful?":"Judgement"}, inplace=True)
X_train_augmented = pd.concat([train_X_original, augmented_data[["reviews"]]], axis=0, ignore_index=True)
y_train_augmented = pd.concat([train_y_original, augmented_data[["Judgement"]]], axis=0, ignore_index=True)

# Data Loader

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import numpy as np
import torch
import logging


def prepare_dataset(dataset):

    sentences = dataset.reviews.values
    labels = np.array(dataset.Judgement)

    # Load the BERT tokenizer.
    logging.info("Loading BERT tokenizer...")
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

    logging.info(f"Original: {sentences[0]}")
    logging.info(f"Tokenized: {tokenizer.tokenize(sentences[0])}")
    logging.info(
        f"Token IDs: {tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0]))}"
    )

    max_len = 0

    # For every sentence...
    for sent in sentences:
        input_ids = tokenizer.encode(sent, add_special_tokens=True)
        max_len = max(max_len, len(input_ids))

    logging.info(f"Max sentence length: {max_len}")

    # Tokenize all of the sentences and map the tokens to their word IDs.
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in sentences:
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
            sent,  # Sentence to encode.
            add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
            max_length=max_len + 10,  # Pad & truncate all sentences.
            padding='max_length',
            return_attention_mask=True,  # Construct attn. masks.
            return_tensors="pt",  # Return pytorch tensors.
            truncation=True,
        )
        # Add the encoded sentence to the list.
        input_ids.append(encoded_dict["input_ids"])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict["attention_mask"])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    # Print sentence 0, now as a list of IDs.
    logging.info(f"Original: {sentences[0]}")
    logging.info(f"Token IDs:{input_ids[0]}")
    logging.info(f"Length of labels: {len(labels)}")

    return input_ids, attention_masks, labels


def get_loader(input_ids, attention_masks, labels, batch_size=32, loader_type="TRAIN"):
    """
    Returns a dataloader for the dataset.

    The DataLoader needs to know our batch size for training.
    For fine-tuning BERT on a specific task, the authors recommend a batch
    size of 16 or 32.
    """
    # Combine the training inputs into a TensorDataset.
    dataset = TensorDataset(input_ids, attention_masks, labels)

    if loader_type == "TRAIN":
        # We'll take training samples in random order.
        dataloader = DataLoader(
            dataset,  # The training samples.
            sampler=RandomSampler(dataset),  # Select batches randomly
            batch_size=batch_size,  # Trains with this batch size.
        )
    else:
        # For validation the order doesn't matter, so we'll just read them sequentially.
        dataloader = DataLoader(
            dataset,  # The validation samples.
            sampler=SequentialSampler(dataset),  # Pull out batches sequentially.
            batch_size=batch_size,  # Evaluate with this batch size.
        )
    return dataloader


# Utilities

In [ ]:
import numpy as np
import datetime
import torch
from dateutil import tz
import logging
import os
import random
from nltk.tokenize import WhitespaceTokenizer
from allennlp.modules.elmo import batch_to_ids
from transformers import BertTokenizer
from datetime import timedelta

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def format_time(elapsed):
    """
    Takes a time in seconds and returns a string hh:mm:ss
    """
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(timedelta(seconds=elapsed_rounded))


def get_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        logging.info(f"There are {torch.cuda.device_count()} GPU(s) available.")
        logging.info(f"We will use the GPU: {torch.cuda.get_device_name(0)}")
    else:
        logging.info("No GPU available, using the CPU instead.")
        device = torch.device("cpu")
    return device


def current_utc_time():
    return datetime.datetime.now().astimezone(tz.tzutc())


# https://github.com/ylhsieh/pytorch-elmo-classification
class Corpus(object):
    def __init__(
        self,
        sentences,
        labels,
        lowercase=False,
        test_size=0.05,
        max_len=-1,
        label_dict=None,
        shuffle=True,
    ):
        self.shuffle = shuffle
        self.lowercase = lowercase
        self.all_sentences = sentences
        self.all_labels = labels
        self.max_len = max_len
        self.label_dict = label_dict
        self.test_size = test_size
        self.test_sentences = []
        self.test_labels = []
        self.train_sentences = []
        self.train_labels = []

        self.split_dataset()
        self.tokenizer = WhitespaceTokenizer()

        self.train = self.tokenize(self.train_sentences, self.train_labels)
        self.test = self.tokenize(self.test_sentences, self.test_labels)

    def split_dataset(self):
        if self.shuffle:
            data = list(zip(self.all_sentences, self.all_labels))
            random.shuffle(data)
            self.all_sentences, self.all_labels = zip(*data)

        ind = int(self.test_size * len(self.all_sentences))
        self.test_sentences = self.all_sentences[:ind]
        self.test_labels = self.all_labels[:ind]
        self.train_sentences = self.all_sentences[ind:]
        self.train_labels = self.all_labels[ind:]

    def tokenize(self, sentences, labels):
        processed_sentences = []
        processed_labels = []
        cropped = 0
        for (sent, label) in zip(sentences, labels):
            if self.lowercase:
                sent = sent.lower().strip()
            else:
                sent = sent.strip()
            sent = self.tokenizer.tokenize(sent)
            if self.max_len > 0:
                if len(sent) > self.max_len:
                    cropped += 1
                sent = sent[: self.max_len]
            if self.label_dict:
                label = self.label_dict[label]
            processed_sentences.append(sent)
            processed_labels.append(label)
        print(f"Number of sentences cropped: {cropped}")

        return list(zip(processed_labels, processed_sentences))


def batchify(data, bsz, shuffle=False, gpu=False):
    if shuffle:
        random.shuffle(data)
    tags, sents = zip(*data)
    nbatch = (len(sents) + bsz - 1) // bsz
    # downsample biggest class
    # sents, tags = balance_tags(sents, tags)

    for i in range(nbatch):

        batch = sents[i * bsz : (i + 1) * bsz]
        batch_tags = tags[i * bsz : (i + 1) * bsz]
        # lengths = [len(x) for x in batch]
        # sort items by length (decreasing)
        # batch, batch_tags, lengths = length_sort(batch, batch_tags, lengths)

        # Pad batches to maximum sequence length in batch
        # find length to pad to

        # maxlen = lengths[0]
        # for b_i in range(len(batch)):
        #     pads = [pad_id] * (maxlen-len(batch[b_i]))
        #     batch[b_i] = batch[b_i] + pads
        # batch = torch.tensor(batch).long()
        batch = batch_to_ids(batch)
        batch_tags = torch.tensor(batch_tags).long()
        # lengths = [torch.tensor(l).long() for l in lengths]

        # yield (batch, batch_tags, lengths)
        yield batch, batch_tags


# Evaluate

In [ ]:
#from utilities import get_device
import torch
import numpy as np
from sklearn.metrics import classification_report
import logging

device = get_device()


def evaluate_model(model, dataloader):
    model.eval()

    logging.info("Starting Evaluation...")
    print("Starting Evaluation...")

    # Tracking variables
    predictions, true_labels = [], []

    # Predict
    for batch in dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)

        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients, saving memory and
        # speeding up prediction
        with torch.no_grad():
            # Forward pass, calculate logit predictions
            outputs = model(
                b_input_ids, token_type_ids=None, attention_mask=b_input_mask
            )

        # Move logits and labels to CPU
        logits = outputs.logits.detach().cpu().numpy()
        label_ids = b_labels.to("cpu").numpy()

        # Store predictions and true labels
        predictions.append(logits)
        true_labels.append(label_ids)

    logging.info("    DONE.")
    print("    DONE.")

    truth = np.hstack((true_labels))
    a = np.vstack((predictions))
    pred = np.argmax(a, axis=1)
    logging.info(f"Prediction: {pred}")
    logging.info(f"Truth: {truth}")

    report = classification_report(truth, pred, output_dict=True)

    logging.info(f"Classification Report: \n {report}")
    print(f"Classification Report: \n {report}")

    return report


def evaluate_elmo(model, data, criterion_ce, gpu=False):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    all_accuracies = 0.0
    all_loss = 0.0
    nbatches = 0.0

    for batch in data:
        nbatches += 1.0
        source, tags = batch
        if gpu:
            source = source.to("cuda")
            tags = tags.to("cuda")
        # output = model(source, lengths)
        output = model(source)
        v_loss = criterion_ce(output, tags)
        max_vals, max_indices = torch.max(output, -1)

        accuracy = torch.mean(max_indices.eq(tags).float()).item()
        all_accuracies += accuracy
        all_loss += v_loss
    return all_accuracies / nbatches, all_loss / nbatches


# Models

In [ ]:
from transformers import BertForSequenceClassification
import logging
import torch.nn as nn
from allennlp.modules.elmo import Elmo


def get_model(name: str = "BERT", **kwargs):

    if name == "BERT":
        # Load BertForSequenceClassification, the pretrained BERT model with a single
        # linear classification layer on top.
        num_labels = kwargs.get("num_labels", 2)
        output_attentions = kwargs.get("output_attentions", False)
        output_hidden_states = kwargs.get("output_hidden_states", False)

        model = BertForSequenceClassification.from_pretrained(
            "bert-base-uncased",
            num_labels=num_labels,  # The number of output labels--2 for binary classification.
            # You can increase this for multi-class tasks.
            output_attentions=output_attentions,  # Whether the model returns attentions weights.
            output_hidden_states=output_hidden_states,  # Whether the model returns all hidden-states.
        )
        params = list(model.named_parameters())
        logging.info(
            "The BERT model has {:} different named parameters.\n".format(len(params))
        )

        logging.info("==== Embedding Layer ====\n")

        for p in params[0:5]:
            logging.info("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

        logging.info("\n==== First Transformer ====\n")

        for p in params[5:21]:
            logging.info("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

        logging.info("\n==== Output Layer ====\n")

        for p in params[-4:]:
            logging.info("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
        return model

    elif name == "ELMO":
        num_labels = kwargs.get("num_labels", 2)
        on_gpu = kwargs.get("on_gpu", True)
        dropout = kwargs.get("dropout", 0.5)
        model = SimpleELMOClassifier(
            label_size=num_labels, use_gpu=on_gpu, dropout=dropout
        )
        return model

    return None


class SimpleELMOClassifier(nn.Module):
    def __init__(self, label_size, use_gpu, dropout):
        super(SimpleELMOClassifier, self).__init__()
        self.use_gpu = use_gpu
        self.dropout = dropout
        options_file = "./models/pre-trained/elmo_2x4096_512_2048cnn_2xhighway_5.5B_options.json"
        weight_file = "./models/pre-trained/elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.hdf5"
        self.elmo = Elmo(
            options_file, weight_file, 1, dropout=dropout, do_layer_norm=False
        )
        # elmo output
        #         Dict with keys:
        #         ``'elmo_representations'``: ``List[torch.Tensor]``
        #             A ``num_output_representations`` list of ELMo representations for the input sequence.
        #             Each representation is shape ``(batch_size, timesteps, embedding_dim)``
        #         ``'mask'``:  ``torch.Tensor``
        #             Shape ``(batch_size, timesteps)`` long tensor with sequence mask.
        self.conv1 = nn.Conv1d(1024, 16, 3)
        self.p1 = nn.AdaptiveMaxPool1d(128)
        self.activation_func = nn.ReLU6()
        self.dropout_l = nn.Dropout(dropout)
        self.hidden2label = nn.Linear(2048, label_size)

    def init_weights(self):
        for name, param in self.hidden2label.named_parameters():
            if "bias" in name:
                nn.init.constant_(param, 0.0)
            elif "weight" in name:
                nn.init.xavier_uniform_(param)
        for name, param in self.conv1.named_parameters():
            if "bias" in name:
                nn.init.constant_(param, 0.0)
            elif "weight" in name:
                nn.init.xavier_uniform_(param)

    def forward(self, sentences):
        elmo_out = self.elmo(sentences)
        x = elmo_out["elmo_representations"][0]
        x = x.transpose(1, 2)
        x = self.conv1(x)
        x = self.activation_func(x)
        x = self.p1(x)
        x = x.view(-1, 2048)
        x = self.dropout_l(x)
        y = self.hidden2label(x)
        return y


# Preprocessing

In [ ]:
from string import digits
import re


# convert text to lower case
def to_lowercase(x):
    if type(x) is str:
        return x.lower()
    elif x is None:
        return str(None)


# remove urls (e.g. www.sap.com https:)
def remove_urls(x):
    URL_FAST_REGEX = "(((https?|ftp|file):\/\/)|www\\.)\\S+"
    return re.sub(URL_FAST_REGEX, " ", x)


# remove tags (e.g. < br> <html>)
def remove_tags(x):
    FILTER_ALL_TAGS = "<[^>]*>"
    return re.sub(FILTER_ALL_TAGS, " ", x)


# remove special characters (eg. ' ! Ã £ Ã ¢ { })
def filter_characters_regex(x):
    return re.sub(r"[^a-zA-Z0-9,.']+", " ", x)


def numbers_and_mail(txt):
    # Remove Number and Email Id
    pretxt = re.sub(r"\S*@\S*\s?|\w*\d\w*", "", str(txt))
    # pretxt = re.sub(r'[\w\.,]+@[\w\.,]+\.\w+','' ,txt)
    remove_digits = str.maketrans("", "", digits)
    pretxt = pretxt.translate(remove_digits)
    return pretxt


# remove multiple whitespaces
def remove_multiple_white_spaces(x):
    FILTER_MULTIPLE_WHITESPACES = "\s\s+"
    return re.sub(FILTER_MULTIPLE_WHITESPACES, " ", x)


# strip spaces at the end and the starting
def strip_white_spaces(x):
    return x.strip()


# invoke all above functions
def preprocess(x):
    return strip_white_spaces(
        remove_multiple_white_spaces(
            numbers_and_mail(
                filter_characters_regex(remove_tags(remove_urls(to_lowercase(x))))
            )
        )
    )


# Train

In [ ]:
import random
import numpy as np
import time
import torch
#from utilities import format_time, get_device, flat_accuracy
import logging

device = get_device()


def train_model(
    model,
    train_dataloader,
    validation_dataloader,
    optimizer,
    scheduler,
    seed=42,
    epochs=5,
):
    """
    This training code is based on the `run_glue.py` script here:
    https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
    """
    # Set the seed value all over the place to make this reproducible.
    seed_val = seed

    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    model = model.to(device)

    # We'll store a number of quantities such as training and validation loss,
    # validation accuracy, and timings.
    training_stats = []

    # Measure the total training time for the whole run.
    total_t0 = time.time()

    # For each epoch...
    for epoch_i in range(0, epochs):

        # Perform one full pass over the training set.

        logging.info("")
        logging.info("======== Epoch {:} / {:} ========".format(epoch_i + 1, epochs))
        logging.info("Training...")
        print("")
        print("======== Epoch {:} / {:} ========".format(epoch_i + 1, epochs))
        print("Training...")

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_train_loss = 0

        # Put the model into training mode. Don't be mislead--the call to
        # `train` just changes the *mode*, it doesn't *perform* the training.
        # `dropout` and `batchnorm` layers behave differently during training
        # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):

            # Progress update every 40 batches.
            if step % 5 == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)

                # Report progress.
                logging.info(
                    "  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.".format(
                        step, len(train_dataloader), elapsed
                    )
                )
                print(
                    "  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.".format(
                        step, len(train_dataloader), elapsed
                    )
                )

            # Unpack this training batch from our dataloader.
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using the
            # `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids
            #   [1]: attention masks
            #   [2]: labels
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            # Always clear any previously calculated gradients before performing a
            # backward pass. PyTorch doesn't do this automatically because
            # accumulating the gradients is "convenient while training RNNs".
            # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
            model.zero_grad()

            # Perform a forward pass (evaluate the model on this training batch).
            # The documentation for this `model` function is here:
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # It returns different numbers of parameters depending on what arguments
            # arge given and what flags are set. For our useage here, it returns
            # the loss (because we provided labels) and the "logits"--the model
            # outputs prior to activation.
            output = model(
                b_input_ids,
                token_type_ids=None,
                attention_mask=b_input_mask,
                labels=b_labels,
            )

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value
            # from the tensor.
            total_train_loss += output.loss.item()

            # Perform a backward pass to calculate the gradients.
            output.loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()

            # Update the learning rate.
            scheduler.step()

        # Calculate the average loss over all of the batches.
        avg_train_loss = total_train_loss / len(train_dataloader)

        # Measure how long this epoch took.
        training_time = format_time(time.time() - t0)

        logging.info("")
        logging.info("  Average training loss: {0:.2f}".format(avg_train_loss))
        #logging.info("  Training epcoh took: {:}".format(training_time))
        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        #print("  Training epcoh took: {:}".format(training_time))

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.

        logging.info("")
        logging.info("Running Validation...")
        print("")
        print("Running Validation...")

        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        model.eval()

        # Tracking variables
        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        # Evaluate data for one epoch
        for batch in validation_dataloader:

            # Unpack this training batch from our dataloader.
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using
            # the `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids
            #   [1]: attention masks
            #   [2]: labels
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            # Tell pytorch not to bother with constructing the compute graph during
            # the forward pass, since this is only needed for backprop (training).
            with torch.no_grad():

                # Forward pass, calculate logit predictions.
                # token_type_ids is the same as the "segment ids", which
                # differentiates sentence 1 and 2 in 2-sentence tasks.
                # The documentation for this `model` function is here:
                # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                # Get the "logits" output by the model. The "logits" are the output
                # values prior to applying an activation function like the softmax.
                output = model(
                    b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels,
                )

            # Accumulate the validation loss.
            total_eval_loss += output.loss.item()

            # Move logits and labels to CPU
            logits = output.logits.detach().cpu().numpy()
            label_ids = b_labels.to("cpu").numpy()

            # Calculate the accuracy for this batch of test sentences, and
            # accumulate it over all batches.
            total_eval_accuracy += flat_accuracy(logits, label_ids)

        # Report the final accuracy for this validation run.
        avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
        logging.info("  Accuracy: {0:.2f}".format(avg_val_accuracy))
        print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

        # Calculate the average loss over all of the batches.
        avg_val_loss = total_eval_loss / len(validation_dataloader)

        # Measure how long the validation run took.
        validation_time = format_time(time.time() - t0)

        logging.info("  Validation Loss: {0:.2f}".format(avg_val_loss))
        logging.info("  Validation took: {:}".format(validation_time))
        print("  Validation Loss: {0:.2f}".format(avg_val_loss))
        print("  Validation took: {:}".format(validation_time))

        # Record all statistics from this epoch.
        training_stats.append(
            {
                "epoch": epoch_i + 1,
                "Training Loss": avg_train_loss,
                "Valid. Loss": avg_val_loss,
                "Valid. Accur.": avg_val_accuracy,
                #"Training Time": training_time,
                "Validation Time": validation_time,
            }
        )

    logging.info("")
    logging.info("Training complete!")
    print("")
    print("Training complete!")

    logging.info(
        "Total training took {:} (h:mm:ss)".format(format_time(time.time() - total_t0))
    )
    print(
        "Total training took {:} (h:mm:ss)".format(format_time(time.time() - total_t0))
    )
    return model, training_stats


def train_elmo(classifier, train_batch, optimizer_, criterion_ce, gpu=False, clip=5.0):
    classifier.train()
    classifier.zero_grad()
    # source, tags, lengths = train_batch
    source, tags = train_batch
    if gpu:
        source = source.to("cuda")
        tags = tags.to("cuda")

    # output: batch x nclasses
    # output = classifier(source, lengths)
    output = classifier(source)
    c_loss = criterion_ce(output, tags)

    c_loss.backward()

    # `clip_grad_norm` to prevent exploding gradient in RNNs / LSTMs
    torch.nn.utils.clip_grad_norm_(classifier.parameters(), clip)
    optimizer_.step()

    total_loss = c_loss.item()

    # probs = F.softmax(output, dim=-1)
    # max_vals, max_indices = torch.max(probs, -1)
    # accuracy = torch.mean(max_indices.eq(tags).float()).item()

    return total_loss


# FastText

## Before augmentation

In [ ]:
import random
import numpy as np
import os
import logging
import torch
import pandas as pd
from imp import reload
import pickle
import fasttext
from sklearn.metrics import classification_report
from datetime import datetime

reload(logging)

# Parameters
seed = 42
epochs = 25
batch_size = 16
learning_rate = 4e-5
#golden_1 = pd.read_excel("./data/P1-Golden.xlsx")
golden_1 = pd.concat([train_X_original, train_y_original], axis=1)
VECTORS_FILEPATH = "./publish/models/pre-trained/crawl-300d-2M.vec"
SAVE_MODEL = True
output_dir = "./publish/models/"

# Set up log file
logging.basicConfig(
    filename=f"{os.getcwd()}/fasttext-p1.log",
    filemode="a",
    format="%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s",
    datefmt="%H:%M:%S",
    level=logging.INFO,
)

if torch.cuda.is_available():
    device = torch.device("cuda")
    logging.info(f"There are {torch.cuda.device_count()} GPU(s) available.")
    logging.info(f"We will use the GPU: {torch.cuda.get_device_name(0)}")
else:
    logging.info("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

# Set the seed value all over the place to make this reproducible.
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Prepare dataset
# Shuffle the DataFrame rows
#golden_1 = golden_1.sample(frac=1)

fasttext_input = []
fasttext_label = []
for i, row in golden_1.iterrows():
    text = row["reviews"]
    label = "__label__" + str(row["Judgement"])
    fasttext_label.append(label)
    fasttext_input.append(label + " " + text)

golden_1["fasttext_input"] = fasttext_input
golden_1["fasttext_label"] = fasttext_label

# Split the dataframe into 95:5 train:validation splits
train_file = open("./data/p1.fasttext.train", "w", encoding="utf-8")
validation_rows = []
# ind = int((len(golden_1)*95)/100)
for i, row in golden_1.iterrows():
    if True:
        train_file.write(row["fasttext_input"] + "\n")
    # else:
    #     validation_rows.append((row["Judgement"], row["reviews"]))
train_file.close()
validation = pd.concat([X_test, y_test], axis=1)
for i, row in validation.iterrows():
  validation_rows.append((row["Judgement"], row["reviews"]))
# ========================================
#               Training
# ========================================

training_stats = []
start_time = datetime.now()

model = fasttext.train_supervised(input="./data/p1.fasttext.train", lr=learning_rate, epoch=epochs,
                          wordNgrams=2, bucket=200000, dim=300, loss='hs',
                          pretrainedVectors=VECTORS_FILEPATH)
train_time = (datetime.now() - start_time).total_seconds()

# ========================================
#               Evaluation
# ========================================
predicted = []
y_test = []
eval_time_start = datetime.now()

for (l, r) in validation_rows:
    y_test.append(l)
    prediction = int(model.predict(r.strip("\n"))[0][0].split("_")[-1])
    predicted.append(prediction)

eval_report = classification_report(y_test, predicted, output_dict=True)
eval_time = (datetime.now() - eval_time_start)

training_stats.append(
    {"evaluation_report": eval_report, "train_time": train_time, "eval_time": eval_time}
)
logging.info(f"Training Stats: \n {training_stats}")
print(f"Evaluation Report: \n {eval_report}")

# Save report
with open("fasttext-p1.pkl", "wb") as f:
    pickle.dump(training_stats, f)

if SAVE_MODEL:
    # Create output directory if needed
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print(f"Saving model to {output_dir}")

    # Save a trained model.
    # They can then be reloaded using fasttext.load_model(PATH)
    model.save_model(f"{output_dir}/p1-fasttext.bin")


Evaluation Report: 
 {'0': {'precision': 0.92, 'recall': 0.6571428571428571, 'f1-score': 0.7666666666666667, 'support': 105}, '1': {'precision': 0.712, 'recall': 0.9368421052631579, 'f1-score': 0.8090909090909091, 'support': 95}, 'accuracy': 0.79, 'macro avg': {'precision': 0.8160000000000001, 'recall': 0.7969924812030076, 'f1-score': 0.7878787878787878, 'support': 200}, 'weighted avg': {'precision': 0.8212, 'recall': 0.79, 'f1-score': 0.7868181818181819, 'support': 200}}
Saving model to ./publish/models/


## After augmentation

In [ ]:
import random
import numpy as np
import os
import logging
import torch
import pandas as pd
from imp import reload
import pickle
import fasttext
from sklearn.metrics import classification_report
from datetime import datetime

reload(logging)

# Parameters
seed = 42
epochs = 25
batch_size = 16
learning_rate = 4e-5
#golden_1 = pd.read_excel("./data/P1-Golden.xlsx")
golden_1 = pd.concat([X_train_augmented, y_train_augmented], axis=1)
VECTORS_FILEPATH = "./publish/models/pre-trained/crawl-300d-2M.vec"
SAVE_MODEL = True
output_dir = "./publish/models/"

# Set up log file
logging.basicConfig(
    filename=f"{os.getcwd()}/fasttext-p1.log",
    filemode="a",
    format="%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s",
    datefmt="%H:%M:%S",
    level=logging.INFO,
)

if torch.cuda.is_available():
    device = torch.device("cuda")
    logging.info(f"There are {torch.cuda.device_count()} GPU(s) available.")
    logging.info(f"We will use the GPU: {torch.cuda.get_device_name(0)}")
else:
    logging.info("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

# Set the seed value all over the place to make this reproducible.
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Prepare dataset
# Shuffle the DataFrame rows
#golden_1 = golden_1.sample(frac=1)

fasttext_input = []
fasttext_label = []
for i, row in golden_1.iterrows():
    text = row["reviews"]
    label = "__label__" + str(row["Judgement"])
    fasttext_label.append(label)
    fasttext_input.append(label + " " + text)

golden_1["fasttext_input"] = fasttext_input
golden_1["fasttext_label"] = fasttext_label

# Split the dataframe into 95:5 train:validation splits
train_file = open("./data/p1.fasttext.train", "w", encoding="utf-8")
validation_rows = []
# ind = int((len(golden_1)*95)/100)
for i, row in golden_1.iterrows():
    if True:
        train_file.write(row["fasttext_input"] + "\n")
    # else:
    #     validation_rows.append((row["Judgement"], row["reviews"]))
train_file.close()
validation = pd.concat([X_test, y_test], axis=1)
for i, row in validation.iterrows():
  validation_rows.append((row["Judgement"], row["reviews"]))
# ========================================
#               Training
# ========================================

training_stats = []
start_time = datetime.now()

model = fasttext.train_supervised(input="./data/p1.fasttext.train", lr=learning_rate, epoch=epochs,
                          wordNgrams=2, bucket=200000, dim=300, loss='hs',
                          pretrainedVectors=VECTORS_FILEPATH)
train_time = (datetime.now() - start_time).total_seconds()

# ========================================
#               Evaluation
# ========================================
predicted = []
y_test = []
eval_time_start = datetime.now()

for (l, r) in validation_rows:
    y_test.append(l)
    prediction = int(model.predict(r.strip("\n"))[0][0].split("_")[-1])
    predicted.append(prediction)

eval_report = classification_report(y_test, predicted, output_dict=True)
eval_time = (datetime.now() - eval_time_start)

training_stats.append(
    {"evaluation_report": eval_report, "train_time": train_time, "eval_time": eval_time}
)
logging.info(f"Training Stats: \n {training_stats}")
print(f"Evaluation Report: \n {eval_report}")

# Save report
with open("fasttext-p1.pkl", "wb") as f:
    pickle.dump(training_stats, f)

if SAVE_MODEL:
    # Create output directory if needed
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print(f"Saving model to {output_dir}")

    # Save a trained model.
    # They can then be reloaded using fasttext.load_model(PATH)
    model.save_model(f"{output_dir}/p1-fasttext.bin")


Evaluation Report: 
 {'0': {'precision': 0.8484848484848485, 'recall': 0.8, 'f1-score': 0.823529411764706, 'support': 105}, '1': {'precision': 0.7920792079207921, 'recall': 0.8421052631578947, 'f1-score': 0.8163265306122449, 'support': 95}, 'accuracy': 0.82, 'macro avg': {'precision': 0.8202820282028203, 'recall': 0.8210526315789474, 'f1-score': 0.8199279711884755, 'support': 200}, 'weighted avg': {'precision': 0.8216921692169217, 'recall': 0.82, 'f1-score': 0.820108043217287, 'support': 200}}
Saving model to ./publish/models/


# ELMO

In [ ]:
import random
import numpy as np
import os
import logging
import torch
import pandas as pd
from imp import reload
import pickle
from sklearn.metrics import classification_report
#from models import get_model
#from utilities import Corpus, batchify
#from train import train_elmo
#from evaluate import evaluate_elmo
import collections
import torch.optim as optim
import torch.nn as nn
from datetime import datetime

reload(logging)

# Parameters
seed = 42
epochs = 25
batch_size = 16
learning_rate = 2e-4
# golden_1 = pd.read_excel("./data/P1-Golden.xlsx")
golden_1 = pd.concat([train_X_original, train_y_original], axis=1)
CUDA = torch.cuda.is_available()
SAVE_MODEL = True
output_dir = "./publish/models/"

# Set up log file
logging.basicConfig(
    filename=f"{os.getcwd()}/elmo-p1.log",
    filemode="a",
    format="%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s",
    datefmt="%H:%M:%S",
    level=logging.INFO,
)

if CUDA:
    device = torch.device("cuda")
    logging.info(f"There are {torch.cuda.device_count()} GPU(s) available.")
    logging.info(f"We will use the GPU: {torch.cuda.get_device_name(0)}")
else:
    logging.info("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

# Set the seed value all over the place to make this reproducible.
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Prepare dataset

corpus = Corpus(sentences=list(golden_1.reviews), labels=list(golden_1.Judgement))
# Print corpus stats
class_counts = collections.Counter([c[0] for c in corpus.train])
print("Train: {}".format(class_counts))
class_counts = collections.Counter([c[0] for c in corpus.test])
print("Test: {}".format(class_counts))

train_data = batchify(corpus.train, batch_size, shuffle=True)
test_data = batchify(corpus.test, batch_size, shuffle=False)

print("Loaded data!")

# Model
model = get_model("ELMO").to(device)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

# Scheduler
learning_rate_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

# Loss
criterion_ce = nn.CrossEntropyLoss()

# Initialize weights
model.init_weights()

start_time = datetime.now()

training_stats = []
niter_global = 0
for epoch in range(1, epochs + 1):
    logging.info(f"Epoch {epoch}/{epochs}...")
    print(f"Epoch {epoch}/{epochs}...")

    # ========================================
    #               Training
    # ========================================
    # loop through all batches in training data
    train_loss = 0
    nbatches = 0
    for train_batch in train_data:
        loss = train_elmo(model, train_batch, optimizer, criterion_ce, gpu=CUDA)
        train_loss += loss
        niter_global += 1
        nbatches += 1
        if niter_global % 10 == 0:
            msg = "Train loss {:.5f}".format(loss)
            print(msg)
            logging.info(msg)

    train_loss = train_loss / nbatches

    # ========================================
    #               Validation
    # ========================================
    with torch.no_grad():
        accuracy, v_loss = evaluate_elmo(model, test_data, criterion_ce, gpu=CUDA)
    msg = "val acc {:.4f}, val loss {:.4f}".format(accuracy, v_loss)
    print(msg)
    logging.info(msg)
    # we use generator, so must re-gen test data
    test_data = batchify(corpus.test, batch_size, shuffle=False)

    # clear cache between epoch
    torch.cuda.empty_cache()
    # decay learning rate
    learning_rate_scheduler.step()
    # shuffle between epochs
    train_data = batchify(corpus.train, batch_size, shuffle=True)

    training_stats.append(
        {
            "epoch": epoch,
            "Training Loss": train_loss,
            "Valid. Loss": v_loss,
            "Valid. Accur.": accuracy,
        }
    )

train_time = (datetime.now() - start_time).total_seconds()

# ========================================
#               Evaluation
# ========================================
model.eval()
y_test = []
predictions = []
eval_time_start = datetime.now()

for batch in test_data:
    source, labels = batch
    y_test += labels.tolist()
    if CUDA:
        source = source.to("cuda")
    output = model(source)
    _, max_indices = torch.max(output, -1)
    predictions += max_indices.tolist()

eval_report = classification_report(y_test, predictions, output_dict=True)
eval_time = (datetime.now() - eval_time_start).total_seconds()

training_stats.append({"evaluation_report": eval_report, "train_time": train_time, "eval_time": eval_time,})
logging.info(f"Training Stats: \n {training_stats}")
print(f"Evaluation Report: \n {eval_report}")

# Save report
with open("elmo-p1.pkl", "wb") as f:
    pickle.dump(training_stats, f)

if SAVE_MODEL:
    # Create output directory if needed
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print(f"Saving model to {output_dir}")

    # Save a trained model.
    # They can then be reloaded using:
    # model = TheModelClass(*args, **kwargs)
    # model.load_state_dict(torch.load(PATH))
    torch.save(model.state_dict(), f"{output_dir}/p1-elmo.bin")


# BERT

## Before Augmentation

In [ ]:
train_X_original = pd.read_csv("./data/original_data_X_train.csv")
train_y_original = pd.read_csv("./data/original_data_y_train.csv")
augmented_data = pd.read_csv("./data/balanced_augmentation_dataset.csv")
X_test = pd.read_csv("./data/original_data_X_test.csv")
y_test = pd.read_csv("./data/original_data_y_test.csv")

augmented_data=augmented_data[["Reviews", "Useful?"]]
augmented_data.rename(columns={"Reviews":"reviews", "Useful?":"Judgement"}, inplace=True)
X_train_augmented = pd.concat([train_X_original, augmented_data[["reviews"]]], axis=0, ignore_index=True)
y_train_augmented = pd.concat([train_y_original, augmented_data[["Judgement"]]], axis=0, ignore_index=True)

In [ ]:
import random
import numpy as np
import os
import logging
import torch
#from utilities import get_device, current_utc_time
import pandas as pd
from imp import reload
#from data_loader import get_loader, prepare_dataset
from transformers import AdamW, get_linear_schedule_with_warmup
#from models import get_model
#from train import train_model
#from evaluate import evaluate_model
import pickle
#from datetime import datetime
from datetime import datetime, date, timedelta


reload(logging)

# Parameters
model_name = "BERT"
seed = 0
epochs = 1
batch_size = 16
learning_rate = 2e-4
epsilon = 1e-8
#golden_1 = pd.read_excel("./data/P1-Golden.xlsx")
golden_1 = pd.concat([train_X_original, train_y_original], axis=1)
validation = pd.concat([X_test, y_test], axis=1)
SAVE_MODEL = True
output_dir = "./publish/models/"

# Set up log file
#current_time = current_utc_time()
logging.basicConfig(
    filename=f"{os.getcwd()}/bert-p1.log",
    filemode="a",
    format="%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s",
    datefmt="%H:%M:%S",
    level=logging.INFO,
)

device = get_device()
# Set the seed value all over the place to make this reproducible.
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Prepare dataset
all_input_ids, all_attention_masks, all_labels = prepare_dataset(golden_1)
val_input_ids, val_attention_masks, val_labels = prepare_dataset(validation)

# Shuffle data and separate evaluation dataset
print("Separating test and train data")
# while True:
#     indices = np.arange(all_input_ids.shape[0])
#     np.random.shuffle(indices)
#     all_input_ids = all_input_ids[indices]
#     all_attention_masks = all_attention_masks[indices]
#     all_labels = all_labels[indices]
#     val_labels = all_labels[:50]

#     # Ensure that we do not have too much bias in validation dataset
#     bias_ratio = np.count_nonzero(val_labels == 1) / np.count_nonzero(val_labels == 0)
#     if 0.9 < bias_ratio < 1.1:
#         val_input_ids = all_input_ids[:50]
#         val_attention_masks = all_attention_masks[:50]
#         break

val_dataloader = get_loader(
    val_input_ids, val_attention_masks, val_labels, batch_size=batch_size, loader_type="VALIDATE"
)

# input_ids = all_input_ids[50:]
# attention_masks = all_attention_masks[50:]
# labels = all_labels[50:]
input_ids = all_input_ids
attention_masks = all_attention_masks
labels = all_labels

logging.info(f"Number of train samples: {len(input_ids)}")
logging.info(f"Number of validation samples: {len(val_input_ids)}")

# Measure the total training time for the whole run.
start_time = datetime.now()

# ========================================
#               Training
# ========================================

# Prepare dataloader
train_dataloader = get_loader(input_ids, attention_masks, labels, batch_size=batch_size)

# model
model = get_model(model_name).to(device)

# Optimizer
optimizer = AdamW(
    model.parameters(),
    lr=learning_rate,  # args.learning_rate - default is 5e-5, our notebook had 2e-5
    eps=epsilon,  # args.adam_epsilon  - default is 1e-8.
)

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,  # Default value in run_glue.py
    num_training_steps=total_steps,
)

model, stats = train_model(
    model, train_dataloader, val_dataloader, optimizer, scheduler, seed=seed, epochs=epochs
)

# ========================================
#               Evaluation
# ========================================
train_time = (datetime.now() - start_time).total_seconds()
eval_time_start = datetime.now()
eval_report = evaluate_model(model, val_dataloader)
eval_time = (datetime.now() - eval_time_start).total_seconds()

training_stats = {
    "train_size": len(labels),
    "val_size": len(val_labels),
    "training_stats": stats,
    "evaluation_report": eval_report,
    "train_time": train_time,
    "eval_time": eval_time,
}

logging.info(f"Training Stats: \n {training_stats}")
print(f"Evaluation Report: \n {eval_report}")

# Save report
with open("bert-p1.pkl", "wb") as f:
    pickle.dump(training_stats, f)

if SAVE_MODEL:
    # Create output directory if needed
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print(f"Saving model to {output_dir}")

    # Save a trained model, configuration and tokenizer using `save_pretrained()`.
    # They can then be reloaded using `from_pretrained()`
    model_to_save = (
        model.module if hasattr(model, "module") else model
    )  # Take care of distributed/parallel training
    model_to_save.save_pretrained(output_dir)


Separating test and train data


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at


======== Epoch 1 / 1 ========
Training...
  Batch     5  of     50.    Elapsed: 0:00:03.
  Batch    10  of     50.    Elapsed: 0:00:06.
  Batch    15  of     50.    Elapsed: 0:00:09.
  Batch    20  of     50.    Elapsed: 0:00:12.
  Batch    25  of     50.    Elapsed: 0:00:15.
  Batch    30  of     50.    Elapsed: 0:00:18.
  Batch    35  of     50.    Elapsed: 0:00:21.
  Batch    40  of     50.    Elapsed: 0:00:24.
  Batch    45  of     50.    Elapsed: 0:00:27.

  Average training loss: 0.49

Running Validation...
  Accuracy: 0.86
  Validation Loss: 0.37
  Validation took: 0:00:03

Training complete!
Total training took 0:00:33 (h:mm:ss)
Starting Evaluation...
    DONE.
Classification Report: 
 {'0': {'precision': 0.9042553191489362, 'recall': 0.8095238095238095, 'f1-score': 0.8542713567839195, 'support': 105}, '1': {'precision': 0.8113207547169812, 'recall': 0.9052631578947369, 'f1-score': 0.8557213930348259, 'support': 95}, 'accuracy': 0.855, 'macro avg': {'precision': 0.857788036932

## After augmentation

In [ ]:
train_X_original = pd.read_csv("./data/original_data_X_train.csv")
train_y_original = pd.read_csv("./data/original_data_y_train.csv")
augmented_data = pd.read_csv("./data/balanced_augmentation_dataset.csv")
X_test = pd.read_csv("./data/original_data_X_test.csv")
y_test = pd.read_csv("./data/original_data_y_test.csv")

augmented_data=augmented_data[["Reviews", "Useful?"]]
augmented_data.rename(columns={"Reviews":"reviews", "Useful?":"Judgement"}, inplace=True)
X_train_augmented = pd.concat([train_X_original, augmented_data[["reviews"]]], axis=0, ignore_index=True)
y_train_augmented = pd.concat([train_y_original, augmented_data[["Judgement"]]], axis=0, ignore_index=True)

In [ ]:
import random
import numpy as np
import os
import logging
import torch
#from utilities import get_device, current_utc_time
import pandas as pd
from imp import reload
#from data_loader import get_loader, prepare_dataset
from transformers import AdamW, get_linear_schedule_with_warmup
#from models import get_model
#from train import train_model
#from evaluate import evaluate_model
import pickle
#from datetime import datetime
from datetime import datetime, date, timedelta


reload(logging)

# Parameters
model_name = "BERT"
seed = 0
epochs = 1
batch_size = 16
learning_rate = 2e-4
epsilon = 1e-8
#golden_1 = pd.read_excel("./data/P1-Golden.xlsx")
golden_1 = pd.concat([X_train_augmented, y_train_augmented], axis=1)
validation = pd.concat([X_test, y_test], axis=1)
SAVE_MODEL = True
output_dir = "./publish/models/"

# Set up log file
#current_time = current_utc_time()
logging.basicConfig(
    filename=f"{os.getcwd()}/bert-p1.log",
    filemode="a",
    format="%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s",
    datefmt="%H:%M:%S",
    level=logging.INFO,
)

device = get_device()
# Set the seed value all over the place to make this reproducible.
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Prepare dataset
all_input_ids, all_attention_masks, all_labels = prepare_dataset(golden_1)
val_input_ids, val_attention_masks, val_labels = prepare_dataset(validation)

# Shuffle data and separate evaluation dataset
print("Separating test and train data")
# while True:
#     indices = np.arange(all_input_ids.shape[0])
#     np.random.shuffle(indices)
#     all_input_ids = all_input_ids[indices]
#     all_attention_masks = all_attention_masks[indices]
#     all_labels = all_labels[indices]
#     val_labels = all_labels[:50]

#     # Ensure that we do not have too much bias in validation dataset
#     bias_ratio = np.count_nonzero(val_labels == 1) / np.count_nonzero(val_labels == 0)
#     if 0.9 < bias_ratio < 1.1:
#         val_input_ids = all_input_ids[:50]
#         val_attention_masks = all_attention_masks[:50]
#         break

val_dataloader = get_loader(
    val_input_ids, val_attention_masks, val_labels, batch_size=batch_size, loader_type="VALIDATE"
)

# input_ids = all_input_ids[50:]
# attention_masks = all_attention_masks[50:]
# labels = all_labels[50:]
input_ids = all_input_ids
attention_masks = all_attention_masks
labels = all_labels

logging.info(f"Number of train samples: {len(input_ids)}")
logging.info(f"Number of validation samples: {len(val_input_ids)}")

# Measure the total training time for the whole run.
start_time = datetime.now()

# ========================================
#               Training
# ========================================

# Prepare dataloader
train_dataloader = get_loader(input_ids, attention_masks, labels, batch_size=batch_size)

# model
model = get_model(model_name).to(device)

# Optimizer
optimizer = AdamW(
    model.parameters(),
    lr=learning_rate,  # args.learning_rate - default is 5e-5, our notebook had 2e-5
    eps=epsilon,  # args.adam_epsilon  - default is 1e-8.
)

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,  # Default value in run_glue.py
    num_training_steps=total_steps,
)

model, stats = train_model(
    model, train_dataloader, val_dataloader, optimizer, scheduler, seed=seed, epochs=epochs
)

# ========================================
#               Evaluation
# ========================================
train_time = (datetime.now() - start_time).total_seconds()
eval_time_start = datetime.now()
eval_report = evaluate_model(model, val_dataloader)
eval_time = (datetime.now() - eval_time_start).total_seconds()

training_stats = {
    "train_size": len(labels),
    "val_size": len(val_labels),
    "training_stats": stats,
    "evaluation_report": eval_report,
    "train_time": train_time,
    "eval_time": eval_time,
}

logging.info(f"Training Stats: \n {training_stats}")
print(f"Evaluation Report: \n {eval_report}")

# Save report
with open("bert-p1.pkl", "wb") as f:
    pickle.dump(training_stats, f)

if SAVE_MODEL:
    # Create output directory if needed
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print(f"Saving model to {output_dir}")

    # Save a trained model, configuration and tokenizer using `save_pretrained()`.
    # They can then be reloaded using `from_pretrained()`
    model_to_save = (
        model.module if hasattr(model, "module") else model
    )  # Take care of distributed/parallel training
    model_to_save.save_pretrained(output_dir)


Separating test and train data


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at


======== Epoch 1 / 1 ========
Training...
  Batch     5  of    159.    Elapsed: 0:00:06.
  Batch    10  of    159.    Elapsed: 0:00:12.
  Batch    15  of    159.    Elapsed: 0:00:18.
  Batch    20  of    159.    Elapsed: 0:00:24.
  Batch    25  of    159.    Elapsed: 0:00:30.
  Batch    30  of    159.    Elapsed: 0:00:36.
  Batch    35  of    159.    Elapsed: 0:00:42.
  Batch    40  of    159.    Elapsed: 0:00:48.
  Batch    45  of    159.    Elapsed: 0:00:54.
  Batch    50  of    159.    Elapsed: 0:01:00.
  Batch    55  of    159.    Elapsed: 0:01:06.
  Batch    60  of    159.    Elapsed: 0:01:12.
  Batch    65  of    159.    Elapsed: 0:01:19.
  Batch    70  of    159.    Elapsed: 0:01:25.
  Batch    75  of    159.    Elapsed: 0:01:31.
  Batch    80  of    159.    Elapsed: 0:01:37.
  Batch    85  of    159.    Elapsed: 0:01:43.
  Batch    90  of    159.    Elapsed: 0:01:50.
  Batch    95  of    159.    Elapsed: 0:01:56.
  Batch   100  of    159.    Elapsed: 0:02:02.
  Batch   105  of

# Model 2: Roberta

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 32.6 MB/s 
     |████████████████████████████████| 182 kB 77.8 MB/s 
     |████████████████████████████████| 7.6 MB 61.4 MB/s 


Code adapted from:

https://colab.research.google.com/github/DhavalTaunk08/NLP_scripts/blob/master/sentiment_analysis_using_roberta.ipynb#scrollTo=7Gpe9D1QHoCd

# After augmentation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:

train_X_original = pd.read_csv("./data/original_data_X_train.csv")
train_y_original = pd.read_csv("./data/original_data_y_train.csv")
augmented_data = pd.read_csv("./data/balanced_augmentation_dataset.csv")
X_test = pd.read_csv("./data/original_data_X_test.csv")
y_test = pd.read_csv("./data/original_data_y_test.csv")

augmented_data=augmented_data[["Reviews", "Useful?"]]
augmented_data.rename(columns={"Reviews":"reviews", "Useful?":"Judgement"}, inplace=True)
X_train_augmented = pd.concat([train_X_original, augmented_data[["reviews"]]], axis=0, ignore_index=True)
y_train_augmented = pd.concat([train_y_original, augmented_data[["Judgement"]]], axis=0, ignore_index=True)

In [ ]:
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
# EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
class ReviewData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.reviews
        self.targets = self.data.Judgement
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
train_y_original["Judgement"]=  train_y_original["Judgement"].astype(int)
y_test["Judgement"]=  y_test["Judgement"].astype(int)
training_set = ReviewData(pd.concat([X_train_augmented, y_train_augmented], axis=1), tokenizer, MAX_LEN)
testing_set = ReviewData(pd.concat([X_test, y_test], axis=1), tokenizer, MAX_LEN)

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 5)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
model = RobertaClass()
model.to(device)

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), e

In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0

    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        #print(_, data)
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")
   
    return 

In [ ]:
training_loader

In [ ]:
EPOCHS = 1
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training Loss per 5000 steps: 1.6757196187973022
Training Accuracy per 5000 steps: 25.0


317it [01:56,  2.73it/s]

The Total Accuracy for Epoch 0: 78.67298578199052
Training Loss Epoch: 0.5465499488901264
Training Accuracy Epoch: 78.67298578199052


In [ ]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accuracy(big_idx, targets)
            all_preds.append(big_idx.cpu().detach().numpy())
            all_targets.append(targets.cpu().detach().numpy())
            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    all_preds = np.concatenate(all_preds)
    all_targets = np.concatenate(all_targets)
    #print(all_preds, all_targets)
    print(classification_report(all_targets, all_preds))
    return epoch_accu


In [ ]:
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

2it [00:00, 19.08it/s]

Validation Loss per 100 steps: 0.18931196630001068
Validation Accuracy per 100 steps: 100.0


50it [00:02, 17.53it/s]

Validation Loss Epoch: 0.2876202328503132
Validation Accuracy Epoch: 89.5
              precision    recall  f1-score   support

           0       0.89      0.91      0.90       105
           1       0.90      0.87      0.89        95

    accuracy                           0.90       200
   macro avg       0.90      0.89      0.89       200
weighted avg       0.90      0.90      0.89       200

Accuracy on test data = 89.50%


# Before data augmentation:

In [ ]:
training_set = ReviewData( pd.concat([train_X_original, train_y_original], axis=1), tokenizer, MAX_LEN)
testing_set = ReviewData(pd.concat([X_test, y_test], axis=1), tokenizer, MAX_LEN)

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

del tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)
del model
model = RobertaClass()
model.to(device)
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)
EPOCHS = 1
for epoch in range(EPOCHS):
    train(epoch)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer 

Training Loss per 5000 steps: 1.6566047668457031
Training Accuracy per 5000 steps: 12.5


100it [00:36,  2.74it/s]

The Total Accuracy for Epoch 0: 68.75
Training Loss Epoch: 0.8903598971664906
Training Accuracy Epoch: 68.75


In [ ]:
acc = valid(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

2it [00:00, 18.57it/s]

Validation Loss per 100 steps: 0.9771075248718262
Validation Accuracy per 100 steps: 50.0


50it [00:02, 17.28it/s]

Validation Loss Epoch: 0.3865887239575386
Validation Accuracy Epoch: 87.0
              precision    recall  f1-score   support

           0       0.96      0.78      0.86       105
           1       0.80      0.97      0.88        95

    accuracy                           0.87       200
   macro avg       0.88      0.87      0.87       200
weighted avg       0.89      0.87      0.87       200

Accuracy on test data = 87.00%


## RoBERTa Analysis

#### (After data augmentation)

In [ ]:
EPOCHS = 1
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training Loss per 5000 steps: 0.16894738376140594
Training Accuracy per 5000 steps: 100.0


317it [01:55,  2.73it/s]

The Total Accuracy for Epoch 0: 89.25750394944708
Training Loss Epoch: 0.27435735361621205
Training Accuracy Epoch: 89.25750394944708


In [ ]:
def valid_and_analyze(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    all_preds = []
    all_targets = []
    all_data = []
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs = model(ids, mask, token_type_ids).squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accuracy(big_idx, targets)
            all_preds.append(big_idx.cpu().detach().numpy())
            all_targets.append(targets.cpu().detach().numpy())
            all_data.append(tokenizer.batch_decode(data["ids"]))
            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%5000==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples
                print(f"Validation Loss per 100 steps: {loss_step}")
                print(f"Validation Accuracy per 100 steps: {accu_step}")
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    all_preds = np.concatenate(all_preds)
    all_targets = np.concatenate(all_targets)
    all_data = np.concatenate(all_data)
    #print(all_preds, all_targets)
    print(classification_report(all_targets, all_preds))
    return epoch_accu, all_preds, all_targets, all_data


In [ ]:
acc, preds, targets, input_sentences = valid_and_analyze(model, testing_loader)
print("Accuracy on test data = %0.2f%%" % acc)

2it [00:00, 11.06it/s]

Validation Loss per 100 steps: 0.7252010703086853
Validation Accuracy per 100 steps: 75.0


50it [00:03, 16.59it/s]

Validation Loss Epoch: 0.2968002664297819
Validation Accuracy Epoch: 89.5
              precision    recall  f1-score   support

           0       0.90      0.90      0.90       105
           1       0.89      0.89      0.89        95

    accuracy                           0.90       200
   macro avg       0.89      0.89      0.89       200
weighted avg       0.90      0.90      0.90       200

Accuracy on test data = 89.50%


In [ ]:
debug_df = pd.DataFrame({'preds':preds, 'targets':targets, 'input_sent':input_sentences})
debug_df = debug_df[debug_df["preds"] != debug_df["targets"]]


In [ ]:
debug_df

,preds,targets,input_sent
3,0,1,<s>This app is wonderful! Quite a lifesaver. T...
23,0,1,<s>With iPhone 5 works just perfectly</s><pad>...
26,1,0,<s>Pac-Man 256 is now one of my favorite games...
30,1,0,<s>It's ok. If you're going to charge for an a...
34,1,0,<s>5 estrellas o can talk we my love one and s...
48,0,1,<s>Great app. Love that I can scan several dif...
52,0,1,<s>Love this app and it's gotten even better w...
55,0,1,<s>I run GTD on my iPhone using the one note a...
65,1,0,<s>Phenomenal!One of my most used apps. I have...
81,0,1,<s>Best program for Twitter This has been the ...
